In [46]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.0.1-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [47]:
#import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
import pandas as pd

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [23]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/DelayedFlights.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("DelayedFlights.csv"), sep=",", header=True)
url_cities='https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/cities500.txt'
spark.sparkContext.addFile(url_cities)
df_lookup_geo = spark.read.csv(SparkFiles.get("cities500.txt"), sep="\t", header=True)

# we are going to do a lookup here as well so upload the airportCodes.csv file from you Resources directory 
df_lookup_city_name=spark.read.csv('airportCodes.csv', sep=',', header=True)

In [10]:
#Create temporary views for each of our dataframes
# We are going to filter the data to US only as we create the Temp Views.

df.createOrReplaceTempView('delayed')
df_lookup_city_name.createOrReplaceTempView('lookup_city')
df_lookup_geo.createOrReplaceTempView('lookup_geo')

In [48]:
# Here is the  initial query presented to you for optimization
# Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|Origin|UniqueCarrier|    Origin_City|Origin_latitude|Origin_Longitude|Dest_latitude|Dest_longitude|delayed|   avgCarrierDelay|
+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|   ATL|           EV|    Atlanta, GA|         33.749|       -84.38798|     31.31129|     -92.44514|   99.0| 28.81025641025641|
|   ABQ|           DL|Albuquerque, NM|       35.08449|      -106.65114|       33.749|     -84.38798|   99.0|40.401869158878505|
|   ATW|           EV|   Appleton, WI|       44.26193|       -88.41538|       33.749|     -84.38798|   99.0|30.546666666666667|
|   BWI|           WN|  Baltimore, MD|       39.29038|       -76.61219|     30.26715|     -97.74306|   95.0|12.741935483870968|
|   CAK|           EV|      Akron, OH|       41.08144|       -81.51901|       33.749|     -84.38798|   9

In [54]:
#partition the largest table
df.write.parquet('parquet_delayed',mode='overwrite')

In [56]:
# read the new parquet formatted data
df=spark.read.parquet('parquet_delayed')

In [57]:
# create a view (same name as before so we don't have change our SQL)
df.createOrReplaceTempView('delayed')

In [58]:
# run 2 after storing the data more appropriately and partitioning

# Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|Origin|UniqueCarrier|    Origin_City|Origin_latitude|Origin_Longitude|Dest_latitude|Dest_longitude|delayed|   avgCarrierDelay|
+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|   ATL|           EV|    Atlanta, GA|         33.749|       -84.38798|     31.31129|     -92.44514|   99.0| 28.81025641025641|
|   ABQ|           DL|Albuquerque, NM|       35.08449|      -106.65114|       33.749|     -84.38798|   99.0|40.401869158878505|
|   ATW|           EV|   Appleton, WI|       44.26193|       -88.41538|       33.749|     -84.38798|   99.0|30.546666666666667|
|   BWI|           WN|  Baltimore, MD|       39.29038|       -76.61219|     30.26715|     -97.74306|   95.0|12.741935483870968|
|   CAK|           EV|      Akron, OH|       41.08144|       -81.51901|       33.749|     -84.38798|   9

In [25]:
# Recall that the default shuffle partitions is 200.  We want to bring that down to a reasonable size for both our data and our Spark cluster
# 4 is reasonable for a free Colab 

In [26]:
# Run 3 after setting the shuffle partitions to a more appropriate number
# Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|Origin|UniqueCarrier|    Origin_City|Origin_latitude|Origin_Longitude|Dest_latitude|Dest_longitude|delayed|   avgCarrierDelay|
+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|   ATL|           EV|    Atlanta, GA|         33.749|       -84.38798|     31.31129|     -92.44514|   99.0| 28.81025641025641|
|   ABQ|           DL|Albuquerque, NM|       35.08449|      -106.65114|       33.749|     -84.38798|   99.0|40.401869158878505|
|   ATW|           EV|   Appleton, WI|       44.26193|       -88.41538|       33.749|     -84.38798|   99.0|30.546666666666667|
|   BWI|           WN|  Baltimore, MD|       39.29038|       -76.61219|     30.26715|     -97.74306|   95.0|12.741935483870968|
|   CAK|           EV|      Akron, OH|       41.08144|       -81.51901|       33.749|     -84.38798|   9

In [25]:
# cache your largest temporary view
# Note: when we use SparkSQL to cache a table, the table is immediately cached (no lazy evaluation), when using Pyspark it will not be cached until an action is ran.


In [24]:
# check that your table is cached 
spark.catalog.isCached("delayed")

False

In [27]:
# Run 4 - after caching driver table
# Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|Origin|UniqueCarrier|    Origin_City|Origin_latitude|Origin_Longitude|Dest_latitude|Dest_longitude|delayed|   avgCarrierDelay|
+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|   ATL|           EV|    Atlanta, GA|         33.749|       -84.38798|     31.31129|     -92.44514|   99.0| 28.81025641025641|
|   ABQ|           DL|Albuquerque, NM|       35.08449|      -106.65114|       33.749|     -84.38798|   99.0|40.401869158878505|
|   ATW|           EV|   Appleton, WI|       44.26193|       -88.41538|       33.749|     -84.38798|   99.0|30.546666666666667|
|   BWI|           WN|  Baltimore, MD|       39.29038|       -76.61219|     30.26715|     -97.74306|   95.0|12.741935483870968|
|   CAK|           EV|      Akron, OH|       41.08144|       -81.51901|       33.749|     -84.38798|   9

In [27]:
# you can even cache a large lookup table.


In [31]:
# Run 5 - caching one of the lookups
#Note the runtime
start_time = time.time()

spark.sql("""
with allColumns
(select 
a.Year,
a.Month,
a.DayofMonth,
a.DayOfWeek,
a.DepTime,
a.CRSDepTime,
a.ArrTime,
a.CRSArrTime,
a.UniqueCarrier,
a.FlightNum,
a.TailNum,
a.ActualElapsedTime,
a.CRSElapsedTime,
a.AirTime,
a.ArrDelay,
a.DepDelay,
a.Origin,
b.City as Origin_City,
geo.latitude as Origin_latitude,
geo.longitude as Origin_longitude,
a.Dest,
c.City as Dest_City,
geo_dest.latitude as Dest_latitude,
geo_dest.longitude as Dest_longitude,
a.Distance,
a.TaxiIn,
a.TaxiOut,
a.Cancelled,
a.CancellationCode,
a.Diverted,
a.CarrierDelay,
a.WeatherDelay,
a.NASDelay,
a.SecurityDelay,
a.LateAircraftDelay from  delayed a 
  inner join lookup_city b
    on a.Origin=b.airportCode
  inner join lookup_city c
    on a.Dest=c.airportCode
  inner join lookup_geo geo
on split(b.City,',')[0]=geo.name
     and trim(split(b.City,',')[1])=geo.admin1_code
  inner join lookup_geo geo_dest
    on c.City=concat(geo_dest.name,', ',geo_dest.admin1_code)
)
select Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude, max(DepDelay) as delayed, avg(CarrierDelay) avgCarrierDelay 
from allColumns 
group by Origin, UniqueCarrier, Origin_City, Origin_latitude, Origin_Longitude, Dest_latitude, Dest_longitude
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|Origin|UniqueCarrier|    Origin_City|Origin_latitude|Origin_Longitude|Dest_latitude|Dest_longitude|delayed|   avgCarrierDelay|
+------+-------------+---------------+---------------+----------------+-------------+--------------+-------+------------------+
|   ATL|           EV|    Atlanta, GA|         33.749|       -84.38798|     31.31129|     -92.44514|   99.0| 28.81025641025641|
|   ABQ|           DL|Albuquerque, NM|       35.08449|      -106.65114|       33.749|     -84.38798|   99.0|40.401869158878505|
|   ATW|           EV|   Appleton, WI|       44.26193|       -88.41538|       33.749|     -84.38798|   99.0|30.546666666666667|
|   BWI|           WN|  Baltimore, MD|       39.29038|       -76.61219|     30.26715|     -97.74306|   95.0|12.741935483870968|
|   CAK|           EV|      Akron, OH|       41.08144|       -81.51901|       33.749|     -84.38798|   9

In [15]:
# run 6 - remove unnecesary columns from the SQL
# Note the runtime
start_time = time.time()

#<put your sql here>

print("--- %s seconds ---" % (time.time() - start_time))

--- 8.487701416015625e-05 seconds ---


In [16]:
# run 7 - filter the lookup tables in the SQL
# Note the runtime
start_time = time.time()

#< re-write your sql applying a filter to each lookup table>

print("--- %s seconds ---" % (time.time() - start_time))

--- 6.842613220214844e-05 seconds ---


In [36]:
#recreate the dataframes selecting only the columns you need, filtering the data before creating the view, then caching the views.
# columns needed are 'UniqueCarrier','DepDelay','Origin','CarrierDelay','Dest' from the main table
#<df.select here>
# filter the df_lookup_city data prior to creating the view to only contain USA data

# filter the df_lookup_geo data prior to creating the view to only contain US data and select only the columns you need to perform the lookup
# fields from geo ('name','latitude','longitude','admin1_code')

df_lookup_city_name.createOrReplaceTempView('lookup_city')
df_lookup_geo.createOrReplaceTempView('lookup_geo')
df.createOrReplaceTempView('delayed')
spark.sql("cache table delayed")
spark.sql("cache table lookup_geo")

DataFrame[]

In [34]:
# run 8 - filtered lookup dataframes
# Note the runtime
start_time = time.time()

#<final query should return the same data as the first query, but much faster>

print("--- %s seconds ---" % (time.time() - start_time))

--- 3.790855407714844e-05 seconds ---


In [30]:
# Remember to uncache the table as soon as you are done.
spark.sql("uncache table delayed")
spark.sql("uncache table lookup_geo")

DataFrame[]

In [31]:
#Verify that the table is no longer cached
if spark.catalog.isCached("delayed") or spark.catalog.isCached("lookup_geo"):
  print("a table is till cached")
else:
  print("all clear")

all clear
